<a href="https://colab.research.google.com/github/boramkim0514/KOIPA_AI/blob/main/Keras_MNIST_%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8_%EC%97%B0%EC%8A%B51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import datasets

In [5]:
mnist = datasets.mnist  #datasets에서 mnist를 찾아서 변수로  생성

(X_train, y_train), (X_test, y_test) = mnist.load_data()  #훈련과 테스트용으로 데이터 세트 나누기

11493376/11490434 [==============================] - 0s 0us/step


In [48]:
X_train.ndim  #dimension  화인  

3

In [12]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
X_train.shape, y_train.shape #train set shape  확인

((60000, 28, 28), (60000,))

In [8]:
X_test.shape, y_test.shape  # test set shape 확인해보고싶어서  (배치  갯수의 변화)

((10000, 28, 28), (10000,))

Fully Connected Layer 를 사용하기 위해 평탄화(필요시 OHE  진행)


#Keras
tensorflow를 기반으로 하는 유명 논문, 레이어 구현해놓은 패키지


# Fully Connected Layer 모델링 준비
    * 레이어들은 tf.keras.layers 패키지에 모두 들어 있음
    * Flatten Layer : 배치를 제외한 평탄화 담당
    * Dense Layer : Affine 연산 담당
    * Activation Layer : 활성화  함수 레이어 (선택에 따라 사용을 안할수도 있음((
    

In [9]:
X_train = X_train /255.0 #정규화 or 평탄화 과정??
X_test = X_test /255.0

#  Fully Connected Layer  모델링
  *  계산그래프 생성
  * Model 객체 생성시 계산그래프 생성

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Activation, Input # keras tensorflow 패캐지에서 관련  layer 불러오기 

input  =  Input(shape=(28, 28)) #Hin, Win에 해당할 듯? input 변수에 Input 함수 레이어에 shape이 28,  28  배열을 넣기?   
net = Flatten()(input)

# 은닉층
net = Dense(512)(net)
net = Activation("relu")(net)

net = Dense(256)(net)
net = Activation("relu")(net)

# 출력층 
net = Dense(10)(net)
net = Activation("softmax")(net)

# 모델 생성 - 계산그래프 세션 내에 생성된다
model = tf.keras.Model(inputs=input, outputs=net, name='BASIC_MNIST')
model.summary() #모델 요약

Model: "BASIC_MNIST"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                

#  최적화 계획 세우기
* Loss Function(손실  함수)  -  MSE, CEE 쓸지를 결정
* Optimization(최적화 함수) - SGD, ADAM, RMSProp을 쓸지를 결정
* Metrics - 테스트 세트에 대한 평가 기준

# Loss Function 선정 기준
  *  이진 분류를 수행하는 경우(Binary Classification)
    *  mse : 출력층의 뉴런이 1개인 경우 -  with sigmoid
    * cross entropy error : 출력층의 뉴런이 2개인 경우 - with softmax(제일 자주 사용되는  방법)
        *  binary_crossentropy를 사용하면 된다
  *  다중 분류를 수행하는 경우 (Multiclass Classification)
    * cross entropy error : 출력층의 뉴런을 클래스의 개수만큼 설정
        *  categorical_crossentropy 를 사용하면 된다

# Categorical Cross Entorpy의 종류
  *  Label이 [0,  1,  2]처럼 OHE이 안된경우, sparse_categorical_crossentropy사용
  *  OHE 된 경우,  categorical_crossentropy  사용   

y_train을 확인해서 어떤 Loss function을 사용할지 결정

In [11]:
y_train.shape

(60000,)

In [17]:
y_train[:3]

array([5, 0, 4], dtype=uint8)

In [19]:
loss_func = tf.keras.losses.sparse_categorical_crossentropy
loss_func

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy>

In [23]:
y_train.shape

(60000,)

In [24]:
# OHE 되어있을 경우
tf.keras.losses.categorical_crossentropy 

<function tensorflow.python.keras.losses.categorical_crossentropy>

In [25]:
#이진분류면 binary_crossentropy
tf.keras.losses.binary_crossentropy 

<function tensorflow.python.keras.losses.binary_crossentropy>

# optimizer 설정
*  SGD - tf.keras.optimizer.SGD()
* rmsprops - tf.keras.optimizer.RMSprop()
* adam - tf.keras.optimizers.Adam() #  일반적으로 제일 많이 쓰는 기법


In [26]:
optm = tf.keras.optimizers.Adam()
optm

# 테스트 세트 평가방법(Metrics)선정
    *  evalute

In [27]:
metrics = ['accuracy']
#metrics = ['acc']
#metrics = [tf.keras.metrics.Accuracy()]

# 모던 컴파일
    *  텐서플로우 세션에 위치한 계산그래프에 데이터를 집어넣기 직전 작업을 완료
    *  손실함수 이진분류 
        *  (출력층 1개: MSE - sigmoid, 출력층 2개 경우:CEE - softmax)binray_crossentropy 사용
    * MSE, CEE(Sparse/categorical_rossentropy),  최적화(SGD, RMSprop,Adam),  평가 방법들(tf.keras.metrics.Accuracy())을 계산그래프에 적용


In [44]:
model.compile(
    optimizer=optm,
    loss=loss_func,
    metrics=metrics
    )

# 훈련용 하이퍼 파라미터 설정
  * 에폭 횟수(num_epochs)
  * 배치  크기(batch_size)

In [45]:
num_epochs = 10
batch_size = 32

# 훈련 시작
    * fit

In [46]:
model.fit(
    X_train, #  훈련 X  데이터(feature)
    y_train, # 훈련 y 레이블(target)
    batch_size=batch_size, #배치 사이즈
    epochs=num_epochs, #에폭 횟수
    shuffle=True,  #데이터를 섞어가면서 훈련
    validation_split=0.2  #검증 세트 비율 
)

Epoch 1/10
1500/1500 [==============================] - 13s 8ms/step - loss: 0.2061 - accuracy: 0.9375 - val_loss: 0.1114 - val_accuracy: 0.9667
Epoch 2/10
1500/1500 [==============================] - 11s 7ms/step - loss: 0.0869 - accuracy: 0.9729 - val_loss: 0.0922 - val_accuracy: 0.9737
Epoch 3/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0544 - accuracy: 0.9820 - val_loss: 0.1294 - val_accuracy: 0.9609
Epoch 4/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0441 - accuracy: 0.9858 - val_loss: 0.1022 - val_accuracy: 0.9736
Epoch 5/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0332 - accuracy: 0.9890 - val_loss: 0.1046 - val_accuracy: 0.9739
Epoch 6/10
1500/1500 [==============================] - 12s 8ms/step - loss: 0.0284 - accuracy: 0.9909 - val_loss: 0.0992 - val_accuracy: 0.9757
Epoch 7/10
1500/1500 [==============================] - 11s 8ms/step - loss: 0.0248 - accuracy: 0.9921 - val_loss: 0.1399 - val_ac

#  런칭 직전 평가
    *  evaluate


In [47]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 4ms/step - loss: 0.1043 - accuracy: 0.9779


[0.10425996035337448, 0.9779000282287598]